# FAKE REVIEW NODE EMBEDDING

In [216]:
# importing all necessary libraries
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import os
import networkx as nx
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
#from node2vec import Node2Vec as n2v
sns.set()

%matplotlib inline

In [217]:
import os, sys
from copy import deepcopy
import numpy as np
import pandas as pd

import torch as th
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [218]:
print(f"Torch version: {th.__version__}.  CUDA version: {th.version.cuda}")

Torch version: 1.12.1+cu113.  CUDA version: 11.3


In [219]:
dgl_version = f"dgl-cu{th.version.cuda.replace('.', '')}==0.7.2"
dgl_version

'dgl-cu113==0.7.2'

In [220]:
# Follow instructions at https://www.dgl.ai/pages/start.html
!{sys.executable} -m pip install {dgl_version} -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [221]:
os.environ['DGLBACKEND'] = 'pytorch'
import dgl
from dgl import function as fn

In [222]:
dgl.__version__

'0.7.2'

In [223]:
from dgl.dataloading.pytorch import NodeDataLoader
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [224]:
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import argparse

from dgl.sampling import node2vec_random_walk

In [225]:
#importing libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import statistics
import math
import os
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('all')
import re
pd.set_option('max_colwidth', 800)


#importing Libraies
import networkx as nx
import matplotlib.pyplot as plt
from numpy import genfromtxt
import collections
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,LogisticRegression, RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [226]:
#loading dataset
df=pd.read_csv('/content/category 8.csv')
df.head()

,category,rating,label,text_
0,Clothing_Shoes_and_Jewelry_5,5.0,CG,Granddaughter loves it. Bought boxer briefs for my daughter and she loved them.
1,Clothing_Shoes_and_Jewelry_5,3.0,CG,I wish the strap was more of a wide width. I normally wear a size 10 and ordered a
2,Clothing_Shoes_and_Jewelry_5,5.0,CG,"Nice, thick material 10 yr old. The only problem is that it's a little too large for"
3,Clothing_Shoes_and_Jewelry_5,5.0,CG,They are easy to use and have the wide width. I will keep them for the summer and they
4,Clothing_Shoes_and_Jewelry_5,5.0,CG,Love them!! Great flippies plus they fit well!!Wife loved it.


In [227]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [228]:
df.shape

(3848, 4)

In [229]:
df['label'].value_counts()

OR    1924
CG    1924
Name: label, dtype: int64

In [230]:
# dropping unnecessary columns. 
df.drop(['category', 'rating'], axis = 1, inplace=True)
df.head()

,label,text_
0,OR,Looks a little clunky and a bit small I would stick to the regular leather chuck Taylors
1,OR,child has marks on her feet after wearing for 2 hours
2,OR,"Nice bra, not great fit (slightly small). Not great support - this isn't an every day bra. I got it for under a specific shirt and for that limited purpose, it's good. If I was ordering again, I would go up one band size."
3,CG,"This is my second pair of these, and they are very comfortable. I have worn them for a couple of hours and they are still warm and comfortable.I bought these for my daughter's birthday and she loves them. They are very comfortable.I bought this for my husband for his birthday. He's a huge fan of the sneaker and it fits him so well. He says it's very comfortable and he's wearing it all the time. I love the arch support. He wears a size 12. He says he likes it, but it's a little tight. It's definitely not for him. But he's happy to get it."
4,CG,"Heavy duty leather belt. Runs small. The only problem is that it's a little too large. Great quality and fit. The front part is very small, but the back is very large."


In [231]:
z = {'OR' : 1, 'CG' : 0}  # 1 for original reviews and 0 for computer generated reviews
df['label'] = df['label'].map(z)

In [232]:
# Cleaning Raw tweets
def clean_text(text):
    
    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])
    
    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove brackets and parantheses (and removing the text inside the brackets and parantheses)
    text = re.sub("[\(\[].*?[\)\]]", "", text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', ' ', text)
    
    #Filter to allow only alphabets and numbers
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    
    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces 
    text = re.sub('\s+', ' ', text)
    
    return text

In [233]:
# applying that into function.
df["clean_text"] = df['text_'].apply(lambda x: clean_text(x))
df.head(10)

,label,text_,clean_text
0,1,Looks a little clunky and a bit small I would stick to the regular leather chuck Taylors,Looks a little clunky and a bit small I would stick to the regular leather chuck Taylors
1,1,child has marks on her feet after wearing for 2 hours,child has marks on her feet after wearing for hours
2,1,"Nice bra, not great fit (slightly small). Not great support - this isn't an every day bra. I got it for under a specific shirt and for that limited purpose, it's good. If I was ordering again, I would go up one band size.",Nice bra not great fit Not great support this isn t an every day bra I got it for under a specific shirt and for that limited purpose it s good If I was ordering again I would go up one band size
3,0,"This is my second pair of these, and they are very comfortable. I have worn them for a couple of hours and they are still warm and comfortable.I bought these for my daughter's birthday and she loves them. They are very comfortable.I bought this for my husband for his birthday. He's a huge fan of the sneaker and it fits him so well. He says it's very comfortable and he's wearing it all the time. I love the arch support. He wears a size 12. He says he likes it, but it's a little tight. It's definitely not for him. But he's happy to get it.",This is my second pair of these and they are very comfortable I have worn them for a couple of hours and they are still warm and comfortable I bought these for my daughter s birthday and she loves them They are very comfortable I bought this for my husband for his birthday He s a huge fan of the sneaker and it fits him so well He says it s very comfortable and he s wearing it all the time I love the arch support He wears a size He says he likes it but it s a little tight It s definitely not for him But he s happy to get it
4,0,"Heavy duty leather belt. Runs small. The only problem is that it's a little too large. Great quality and fit. The front part is very small, but the back is very large.",Heavy duty leather belt Runs small The only problem is that it s a little too large Great quality and fit The front part is very small but the back is very large
5,0,"Received this bag 5/10/2018. It has the wide width and is a nice bag for the price. It is not the best bag, but it is the most versatile. I would definitely recommend it. It will be a great addition to my collection.I like the shape",Received this bag It has the wide width and is a nice bag for the price It is not the best bag but it is the most versatile I would definitely recommend it It will be a great addition to my collection I like the shape
6,1,Made in USA. Wanted to like them but they run small.,Made in USA Wanted to like them but they run small
7,0,"Fit like a typical size 13.5. The fit is nice, the heel is just a little short. The front of the shoe is slightly too tight to take it off",Fit like a typical size The fit is nice the heel is just a little short The front of the shoe is slightly too tight to take it off
8,0,Always know when we order a size up. We've had a lot of success with them.,Always know when we order a size up We ve had a lot of success with them
9,0,Good BOA shoe for the price. The wide toe box is perfect. The leather is strong and durable.,Good BOA shoe for the price The wide toe box is perfect The leather is strong and durable


In [234]:
# dropping the uncleaned text column "text"
df = df.drop(['text_'], axis=1)

In [235]:
# renaming the column
df.rename(columns = {'clean_text':'text', 'label': 'Class'}, inplace = True)
df.head()

,Class,text
0,1,Looks a little clunky and a bit small I would stick to the regular leather chuck Taylors
1,1,child has marks on her feet after wearing for hours
2,1,Nice bra not great fit Not great support this isn t an every day bra I got it for under a specific shirt and for that limited purpose it s good If I was ordering again I would go up one band size
3,0,This is my second pair of these and they are very comfortable I have worn them for a couple of hours and they are still warm and comfortable I bought these for my daughter s birthday and she loves them They are very comfortable I bought this for my husband for his birthday He s a huge fan of the sneaker and it fits him so well He says it s very comfortable and he s wearing it all the time I love the arch support He wears a size He says he likes it but it s a little tight It s definitely not for him But he s happy to get it
4,0,Heavy duty leather belt Runs small The only problem is that it s a little too large Great quality and fit The front part is very small but the back is very large


In [236]:
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

{'the', 'too', 'either', 'towards', 'elsewhere', 'with', 'next', 'has', 'whoever', 'several', '’d', 'something', 'take', 'then', 'whereas', 'becoming', 'were', 'eleven', 'amongst', '‘s', 'yourself', 'someone', "'m", 'does', 'twenty', 'whereupon', 'latterly', 'serious', 'else', 'namely', 'part', 'they', 'various', 'are', 'hundred', 'onto', 'whither', 'ourselves', 'which', 'one', 'perhaps', 'up', 'no', 'due', 'not', 'fifty', 'where', 'sometime', 'yours', 'nobody', 'another', 'twelve', 'seemed', 'toward', 'across', 'four', 'n’t', 'whereby', 'on', 'very', 'during', 'everyone', 'nothing', 'above', 'therein', 'herein', "'s", 'through', 'full', 'otherwise', 'ca', 'we', 'few', '‘m', 'each', 'wherein', 'whom', 'beside', 'thru', 'third', 'what', 'my', 'about', 'enough', 'anyway', 'sometimes', 'front', 'here', 'had', 'to', 'put', 'first', 'by', 'it', 'yourselves', 'name', 'who', 'everything', 'more', 'give', 'whence', 'latter', 'though', 'ever', 'throughout', 'under', 'become', 'back', 'go', 'onl

In [237]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
print(sw_nltk)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [238]:
sw_nltk.extend(['th','stay'])
print(len(sw_nltk))

181


In [239]:
# preprocessing the data (removing stopwords, convert the sentence to lower case, and also applying lemmatization to get meaningful words )
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
#stop_words = set(stopwords.words('english'))
ps=PorterStemmer()
wnl=WordNetLemmatizer()
def clean_lemma(review):
    review = review.lower()
    review = review.split()
    review = [wnl.lemmatize(word) for word in review if word not in sw_nltk]
    review = [wnl.lemmatize(word) for word in review if word not in sw_spacy]
    review = ' '.join(review)
    return review

In [240]:
# using lemmatization
df['text']=df['text'].apply(clean_lemma)
df.head()

,Class,text
0,1,look little clunky bit small stick regular leather chuck taylor
1,1,child mark foot wearing hour
2,1,nice bra great fit great support day bra got specific shirt limited purpose good ordering band size
3,0,second pair comfortable worn couple hour warm comfortable bought daughter birthday love comfortable bought husband birthday huge fan sneaker fit comfortable wearing time love arch support wear size like little tight definitely happy
4,0,heavy duty leather belt run small problem little large great quality fit small large


### FEATURE EXTRACTION

In [241]:
x = df['text']
y = df['Class']

In [242]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer()
corpii=cv.fit_transform(x) 

In [243]:
feature_names = cv.get_feature_names()
#feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [244]:
len(feature_names)

5883

In [245]:
u1 = pd.DataFrame(corpii.toarray(), columns=feature_names)
u1.head()

,aa,abd,ability,able,abrasion,absolute,absolutely,absolutley,absorb,absorbent,...,zigzag,zip,zipper,zippered,zipping,zombie,zoo,zqwxpctg,zumba,zumiez
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### FEATURE SELECTION

In [246]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 500)
X_kbest_features = chi2_features.fit_transform(u1, y)

In [247]:
chi_support = chi2_features.get_support()

In [248]:
chi_feature = u1.loc[:,chi_support].columns.tolist()

In [249]:
chi_feature

['actual',
 'actually',
 'add',
 'additional',
 'adidas',
 'adjust',
 'adjustable',
 'affordable',
 'age',
 'air',
 'allow',
 'allows',
 'amazing',
 'amazon',
 'area',
 'asian',
 'ask',
 'attach',
 'average',
 'away',
 'baby',
 'bag',
 'baggy',
 'ball',
 'bargain',
 'beat',
 'bed',
 'bend',
 'better',
 'bikini',
 'binding',
 'birthday',
 'bit',
 'blue',
 'body',
 'bonus',
 'boob',
 'bottle',
 'bought',
 'bounce',
 'bow',
 'box',
 'bra',
 'bracelet',
 'brand',
 'brim',
 'buckle',
 'bulge',
 'bulky',
 'bust',
 'buyer',
 'buying',
 'came',
 'cape',
 'care',
 'carhartt',
 'carrying',
 'case',
 'casual',
 'cat',
 'cause',
 'center',
 'certainly',
 'chain',
 'cheap',
 'cheaper',
 'check',
 'chest',
 'child',
 'choice',
 'clean',
 'cleaner',
 'clip',
 'clog',
 'cloth',
 'clothes',
 'clothing',
 'collection',
 'color',
 'colored',
 'colorful',
 'comfortable',
 'coming',
 'compared',
 'complete',
 'completely',
 'con',
 'consider',
 'constantly',
 'construction',
 'continue',
 'control',
 'corn

In [250]:
X_kbest_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [251]:
u3 = pd.DataFrame(X_kbest_features, columns = chi_feature)
u3.head()

,actual,actually,add,additional,adidas,adjust,adjustable,affordable,age,air,...,workout,worn,wrangler,wrap,wrinkle,wrong,xl,yes,zip,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.16755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [252]:
#from scipy.spatial.distance import cdist
#s = cdist(u3, u3, 'euclid')
#s

In [253]:
#X1=pd.DataFrame(s)
#X1

In [254]:
# taking average
y = np.average(u3)
y

0.0036887774261649482

In [255]:
adjmat_df=u3.copy()

In [256]:
adjmat_df[adjmat_df < y]=0
adjmat_df[adjmat_df >= y]=1

In [257]:
adjmat_df

,actual,actually,add,additional,adidas,adjust,adjustable,affordable,age,air,...,workout,worn,wrangler,wrap,wrinkle,wrong,xl,yes,zip,zipper
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [258]:
q = adjmat_df.to_numpy()
q

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [259]:
# SOURCE : https://github.com/dmlc/dgl/issues/3364
src, dst = np.nonzero(X_kbest_features)

In [260]:
src

array([   0,    0,    0, ..., 3847, 3847, 3847])

In [261]:
dst

array([ 32, 230, 234, ..., 386, 463, 471])

In [262]:
g = dgl.graph((src, dst))

In [263]:
n_nodes = g.num_nodes()
n_nodes

3848

In [264]:
nodes = g.nodes()
nodes

tensor([   0,    1,    2,  ..., 3845, 3846, 3847])

In [265]:
n_nodes1 = g.number_of_nodes()
n_nodes1

3848

In [266]:
g.ndata['feat'] = torch.tensor(X_kbest_features)
g.ndata['feat'] 

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [267]:
n_train = int(n_nodes1 * 0.6)
n_val = int(n_nodes1 * 0.2)

In [268]:
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(train_mask)
train_mask

tensor([False, False, False,  ..., False, False, False])

In [269]:
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(val_mask)
val_mask

tensor([False, False, False,  ..., False, False, False])

In [270]:
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
len(test_mask)
test_mask

tensor([False, False, False,  ..., False, False, False])

In [271]:
train_mask[:n_train] = True

In [272]:
val_mask[n_train:n_train + n_val] = True

In [273]:
test_mask[n_train + n_val:] = True

In [274]:
g.ndata['train_mask'] = train_mask

In [275]:
g.ndata['val_mask'] = val_mask

In [276]:
g.ndata['test_mask'] = test_mask

In [277]:
s_array = df['Class'].to_numpy()
s_array

array([1, 1, 1, ..., 0, 1, 1])

In [278]:
t1 = torch.tensor(s_array)
t1

tensor([1, 1, 1,  ..., 0, 1, 1])

In [279]:
g.ndata["label"] =t1

In [280]:
label = g.ndata["label"]

In [281]:
label[train_mask]

tensor([1, 1, 1,  ..., 1, 1, 0])

In [282]:
label[val_mask]

tensor([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,

In [283]:
label[test_mask]

tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0,

In [284]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [285]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [286]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [287]:
clf = LogisticRegression()
clf.fit(g.ndata['feat'][train_mask], label[train_mask])

#y_hat_val_lr = clf.predict_proba(X_kbest_features[val_mask,:])
y_val_lr = clf.predict(g.ndata['feat'][val_mask])

In [288]:
accuracy_score(label[val_mask], y_val_lr)

0.8426527958387516

In [289]:
clf_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
clf_balanced.fit(g.ndata['feat'][train_mask], label[train_mask])

y_val_lrb = clf_balanced.predict(g.ndata['feat'][val_mask])
accuracy_score(label[val_mask], y_val_lrb)

0.8426527958387516

In [290]:
rf = RandomForestClassifier(n_estimators=400)
rf.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = rf.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8322496749024707

In [291]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=1.0, max_depth=1, random_state=0)
gb.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = gb.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8088426527958388

In [292]:
ab = AdaBoostClassifier(n_estimators=400, random_state=0)
ab.fit(g.ndata['feat'][train_mask,:], label[train_mask])

y_val_rf = ab.predict(g.ndata['feat'][val_mask,:])
accuracy_score(label[val_mask], y_val_rf)

0.8101430429128739

In [293]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
cv = KFold(n_splits=10, random_state=7, shuffle=True)

In [294]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter=10000) 
scores = cross_val_score(log, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8552468885281386


In [295]:
from sklearn.svm import LinearSVC
lsvm = LinearSVC()
scores = cross_val_score(lsvm, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8716199945887446


In [296]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa=PassiveAggressiveClassifier(max_iter=1000, random_state=7, tol=1e-3)
scores = cross_val_score(pa, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8323816287878788


In [297]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=300)
scores = cross_val_score(rf, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8497916666666667


In [298]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier(n_estimators=300)
scores = cross_val_score(ab, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8355025703463204


In [299]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=300)
scores = cross_val_score(gb, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8492654220779221


In [300]:
import xgboost as xgb
xgbc = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, 
              gamma=0,  importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, 
             n_estimators=200, n_jobs=16,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, subsample=1,
              tree_method='exact', use_label_encoder=False
              )
scores = cross_val_score(xgbc, g.ndata['feat'], label, cv=cv, n_jobs=-1)
print(np.mean(scores))

0.8552502705627706


In [310]:
# Contruct a two-layer GNN model
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        return h

In [311]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label']
train_mask = g.ndata['train_mask']
valid_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [312]:
node_features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [313]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [314]:
model = SAGE(in_feats=n_features, hid_feats=100, out_feats=n_labels)
opt = torch.optim.Adam(model.parameters())

for epoch in range(100):
    model.train()
    # forward propagation by using all nodes
    logits = model(g, node_features)
    # compute loss
    loss = F.cross_entropy(logits[train_mask], node_labels[train_mask])
    # compute validation accuracy
    acc = evaluate(model, g, node_features, node_labels, test_mask)
    print(acc)
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    #print(loss.item())

    # Save model if necessary.  Omitted in this example.

0.503242542153048
0.5473411154345007
0.5745784695201037
0.6095979247730221
0.6459143968871596
0.6666666666666666
0.6705577172503242
0.6796368352788587
0.6939040207522698
0.7107652399481194
0.7237354085603113
0.7276264591439688
0.7341115434500648
0.7431906614785992
0.748378728923476
0.7587548638132295
0.7652399481193255
0.7769130998702983
0.7808041504539559
0.7846952010376135
0.7898832684824902
0.795071335927367
0.7989623865110247
0.8041504539559015
0.8054474708171206
0.8093385214007782
0.8132295719844358
0.8132295719844358
0.814526588845655
0.8249027237354085
0.8287937743190662
0.8326848249027238
0.8391699092088197
0.8417639429312581
0.8430609597924773
0.8469520103761349
0.8495460440985733
0.8469520103761349
0.8534370946822308
0.8534370946822308
0.8586251621271076
0.8599221789883269
0.8625162127107653
0.8638132295719845
0.8638132295719845
0.8625162127107653
0.8625162127107653
0.8651102464332037
0.867704280155642
0.8664072632944229
0.8690012970168612
0.8690012970168612
0.869001297016861